<a href="https://colab.research.google.com/github/mathewpolonsky/Marketplace-Item-Matching/blob/main/Training_Distilbert_Duplicates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

In [ ]:
!gdown 1-vKglR5qwkmXGT8EK2La2eiNy1ApKLPD
!unzip merged_datasets4nlp.zip
!mv merged_datasets4nlp datasets

Downloading...
From: https://drive.google.com/uc?id=1-vKglR5qwkmXGT8EK2La2eiNy1ApKLPD
To: /content/merged_datasets4nlp.zip
100% 225M/225M [00:04<00:00, 45.6MB/s]
Archive:  merged_datasets4nlp.zip
   creating: merged_datasets4nlp/
  inflating: merged_datasets4nlp/merged_test4nlp.parquet  
  inflating: merged_datasets4nlp/merged_train4nlp.parquet  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


---

In [ ]:
from datetime import datetime

import pandas as pd
import numpy as np

import torch
from torch.utils.data import DataLoader

from sentence_transformers import SentenceTransformer, losses, InputExample, evaluation, models, util

from sklearn.model_selection import train_test_split

In [ ]:
model = SentenceTransformer('sentence-transformers/quora-distilbert-multilingual')

In [ ]:
data = pd.read_parquet('datasets/merged_train4nlp.parquet')

In [ ]:
train_data, val_data = train_test_split(
    data, random_state=42, test_size=.1
)

In [ ]:
data


,target,variantid1,variantid2,full_text_1,full_text_2
0,0.0,51197862,51198054,name: Удлинитель TDM Electric Люкс УЛ05В 5 м (...,name: Удлинитель TDM Electric Люкс УЛ05В 1.5 м...
1,0.0,51197862,51199884,name: Удлинитель TDM Electric Люкс УЛ05В 5 м (...,name: Удлинитель TDM Electric Люкс УЛ05В 3 м (...
2,1.0,53062686,536165289,name: Картридж лазерный Комус 729 (4370B002) ч...,name: Картридж лазерный Комус 729 (4368B002) п...
3,1.0,53602615,587809782,name: Картридж лазерный Комус 729 (4368B002) п...,name: Картридж лазерный Комус 729 (4370B002) ч...
4,1.0,53602615,615149925,name: Картридж лазерный Комус 729 (4368B002) п...,name: Картридж лазерный Комус 729 (4368B002) п...
...,...,...,...,...,...
306535,0.0,810680230,820119986,"name: Комплект 4 шт, Картридж лазерный NV Prin...","name: Комплект 2 шт, Картридж лазерный NV Prin..."
306536,0.0,812434186,815345877,name: Для iPhone 7 плюс 8 плюс военная броня г...,name: Для iPhone 7 8 военная броня корпус гибр...
306537,0.0,815637954,817550808,name: Карта памяти 64 ГБ\ncat3: Карты памяти и...,name: Карта памяти 1 ТБ\ncat3: Карты памяти и ...
306538,0.0,817327230,822083612,"name: Смартфон Mate48 Pro.. 8/256 ГБ, зеленый\...","name: Смартфон Mate48 Pro.. 10/512 ГБ, зеленый..."


In [ ]:
train = train_data.apply(
    lambda x: InputExample(texts=[x['full_text_1'], x['full_text_2']], label=x['target']), axis=1
    ).tolist()

val = val_data['full_text_1'].tolist(), val_data['full_text_2'].tolist(), val_data['target'].tolist()

In [ ]:
train_dataloader = DataLoader(train, shuffle=True, batch_size=16)

evaluator = evaluation.BinaryClassificationEvaluator(*val)

In [ ]:
num_epochs = 6
warmup_steps = 1000
model_save_path = 'drive/MyDrive/models/distilbert-quora-full'
distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE
margin = .5

train_loss = losses.OnlineContrastiveLoss(model=model, distance_metric=distance_metric, margin=margin)

In [ ]:
model = torch.compile(model)

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path=model_save_path
)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15020 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15020 [00:00<?, ?it/s]

### Making Train for Catboost

In [ ]:
test = pd.read_parquet('datasets/merged_train4nlp.parquet')

In [ ]:
test

,variantid1,variantid2,full_text_1,full_text_2
0,52076340,290590137,name: Батарейка AAA щелочная Perfeo LR03/10BL ...,name: Батарейка AAA щелочная Perfeo LR03/2BL m...
1,64525522,204128919,"name: Смартфон Ulefone Armor X5 3/32 ГБ, черны...","name: Смартфон Ulefone Armor X3 2/32 ГБ, черны..."
2,77243372,479860557,name: Цифровой кабель TV-COM HDMI 1.4 (M/ M) F...,name: Кабель HDMI 1.4 (Male/Male) (CG150S-1.5...
3,86065820,540678372,name: Игровая мышь проводная A4Tech Bloody P93...,"name: Мышь A4Tech Bloody P93s Bullet, серый, о..."
4,91566575,258840506,"name: Смартфон Vivo Y93 1815 3/32 ГБ, черный\n...","name: Смартфон Vivo Y81 3/32 ГБ, черный\ncolor..."
...,...,...,...,...
18079,666998614,667074522,name: Оперативная память Corsair VENGEANCE LPX...,name: Оперативная память Corsair VENGEANCE LPX...
18080,670036240,670048449,name: Зарядное устройство для HP 15-bw661ur бл...,name: Зарядное устройство для HP 15s-fq2066ur ...
18081,670284509,684323809,name: Ремешок из натуральной кожи 20 мм для Sa...,name: Ремешок для часов UTHAI Z08 plus из нату...
18082,692172005,704805270,name: Лазерный новогодний проектор для дома и ...,name: Новогодний проектор гирлянда морозостойк...


In [ ]:
text1_embed = model.encode(test["full_text_1"])
text2_embed = model.encode(test["full_text_2"])

In [ ]:
cos_sim = np.empty(text1_embed.shape[0])
for i in range(text1_embed.shape[0]):
    cos_sim[i] = util.cos_sim(text1_embed[i], text2_embed[i]).cpu().numpy()
    
cos_sim

array([0.31175342, 0.76829022, 0.50458109, ..., 0.40414935, 0.93997765,
       0.69510019])

In [ ]:
test["text_embed_cos_sim"] = cos_sim
test

In [ ]:
test[["text_embed_cos_sim", "variantid1", "variantid2"]].to_csv("drive/MyDrive/datasets/train_cb_distilbert_quora_full_6_eps.csv", index=False)